<a href="https://colab.research.google.com/github/markmaloba/DSCORE-Week-6-Independent-Project/blob/master/DSCore_W6_IP_Mark_Maloba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Background

### The context 
You have been recruited as a football analyst in a company - Mchezopesa Ltd and tasked to predict results of a game between team 1 and team 2, based on who's home and who's away, and on whether or not the game is friendly (include rank in your training).

### Metric for success 
We need the model to predict games at an industry-accepted accuracy level (70% at least). In the context of goals, that means our testing results should not vary from the training data by more than +/- 1 goal.

### Design choices 
For this task, I will use a linear regression model as the baseline, to be used for comparison with a second polynomial regression model. This will help us as we rigorously test the model for accuracy, and will "provide a larger net" for catching errors.

### Appropriateness of the available data to answer the given question
I think the data provided is very appropriate both in terms of scope (between the two datasets we have scores, team/ country names, ranking for different teams, match/ tournament type) and volume (we have thousands of records, which means we have better chances of training our models and mitigating (not eliminating) bias).

Although on the topic of bias, I am of the opinion that this type of data is not biased at all. This is because what we have are records of OBSERVATIONS, rather than opinions, and the recordings are provided in their entirety, regardless of tournament, country etc.

Even the way rankings are determined is very mathematical. You play games, you win or lose, and you move up and down the relevant tables based on that.

In [0]:
#Load all necessary libraries & prerequisites

import numpy as np
import pandas as pd
import seaborn as sb
import scipy as sp

from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
ranking = pd.read_csv('fifa_ranking.csv')
results = pd.read_csv('results.csv')

In [0]:
ranking.head(10)

,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1,Germany,GER,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
1,2,Italy,ITA,0.0,57,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
2,3,Switzerland,SUI,0.0,50,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
3,4,Sweden,SWE,0.0,55,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
4,5,Argentina,ARG,0.0,51,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
5,6,Republic of Ireland,IRL,0.0,54,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
6,7,Russia,RUS,0.0,52,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
7,8,Brazil,BRA,0.0,55,-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
8,9,Norway,NOR,0.0,49,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08
9,10,Denmark,DEN,0.0,51,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1993-08-08


In [0]:
results.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
5,1876-03-25,Scotland,Wales,4,0,Friendly,Glasgow,Scotland,False
6,1877-03-03,England,Scotland,1,3,Friendly,London,England,False
7,1877-03-05,Wales,Scotland,0,2,Friendly,Wrexham,Wales,False
8,1878-03-02,Scotland,England,7,2,Friendly,Glasgow,Scotland,False
9,1878-03-23,Scotland,Wales,9,0,Friendly,Glasgow,Scotland,False


For our polynomial analysis, the following data is required for training:
Rank of home team

1. Home team
2. Away team
3. Home team score
4. Away team score
5. Rank of home team
6. Rank of away team
7. Tournament type

Everything else can be discarded for the purposes of our test


In [0]:
#Check columns in both tables to better assess missing parts in each data set, and merging possibilities
print(results.columns)
print(ranking.columns)

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral'],
      dtype='object')
Index(['rank', 'country_full', 'country_abrv', 'total_points',
       'previous_points', 'rank_change', 'cur_year_avg',
       'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
       'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg',
       'three_year_ago_weighted', 'confederation', 'rank_date'],
      dtype='object')


# Merging & Cleaning

In [0]:
#Merging the 2 datasets

merge_df = results.merge(ranking, left_on = ['date', 'home_team'],
                        right_on = ['rank_date', 'country_full'], how = 'inner')
merge_df.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,rank,country_full,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,59,Bolivia,BOL,0.0,13,28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
1,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False,8,Brazil,BRA,0.0,55,-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
2,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False,35,Ecuador,ECU,0.0,23,30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
3,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False,65,Guinea,GUI,0.0,23,-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1993-08-08
4,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,67,Paraguay,PAR,0.0,22,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
5,1993-08-08,Peru,Colombia,0,1,FIFA World Cup qualification,Lima,Peru,False,70,Peru,PER,0.0,16,8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
6,1993-08-08,Zimbabwe,Eswatini,2,0,Friendly,Harare,Zimbabwe,False,50,Zimbabwe,ZIM,0.0,27,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1993-08-08
7,1993-09-23,Saudi Arabia,Costa Rica,1,2,Friendly,Khobar,Saudi Arabia,False,44,Saudi Arabia,KSA,0.0,34,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,AFC,1993-09-23
8,1994-03-15,Gabon,DR Congo,2,1,Friendly,Libreville,Gabon,False,61,Gabon,GAB,0.0,29,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CAF,1994-03-15
9,1994-03-15,Israel,Ukraine,1,0,Friendly,Haifa,Israel,False,54,Israel,ISR,0.0,30,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,UEFA,1994-03-15


In [0]:
# Drop unnecessary columns
# Rename rank column
merge_df.rename(columns = {'rank' : 'home_rank'}, inplace = True)
print(merge_df.columns)

Index(['date', 'home_team', 'away_team', 'home_score', 'away_score',
       'tournament', 'city', 'country', 'neutral', 'home_rank', 'country_full',
       'country_abrv', 'total_points', 'previous_points', 'rank_change',
       'cur_year_avg', 'cur_year_avg_weighted', 'last_year_avg',
       'last_year_avg_weighted', 'two_year_ago_avg', 'two_year_ago_weighted',
       'three_year_ago_avg', 'three_year_ago_weighted', 'confederation',
       'rank_date'],
      dtype='object')


In [0]:
# Drop unnecessary columns
merge_df.drop(columns = ['country_abrv', 'total_points', 'previous_points', 'rank_change', 
                        'cur_year_avg', 'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
                        'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg', 
                        'three_year_ago_weighted'], axis = 1, inplace = True)

Let's see how manageable our dataset is now

In [0]:
merge_df.head(7)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,home_rank,country_full,confederation,rank_date
0,1993-08-08,Bolivia,Uruguay,3,1,FIFA World Cup qualification,La Paz,Bolivia,False,59,Bolivia,CONMEBOL,1993-08-08
1,1993-08-08,Brazil,Mexico,1,1,Friendly,Maceió,Brazil,False,8,Brazil,CONMEBOL,1993-08-08
2,1993-08-08,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Quito,Ecuador,False,35,Ecuador,CONMEBOL,1993-08-08
3,1993-08-08,Guinea,Sierra Leone,1,0,Friendly,Conakry,Guinea,False,65,Guinea,CAF,1993-08-08
4,1993-08-08,Paraguay,Argentina,1,3,FIFA World Cup qualification,Asunción,Paraguay,False,67,Paraguay,CONMEBOL,1993-08-08
5,1993-08-08,Peru,Colombia,0,1,FIFA World Cup qualification,Lima,Peru,False,70,Peru,CONMEBOL,1993-08-08
6,1993-08-08,Zimbabwe,Eswatini,2,0,Friendly,Harare,Zimbabwe,False,50,Zimbabwe,CAF,1993-08-08


In [0]:
merge_df.drop(columns = ['city', 'rank_date', 'confederation'], axis = 1, inplace = True)

In [0]:
merge_df.drop(columns = ['date'], axis = 1, inplace = True)

In [0]:
merge_df.head(7)

,home_team,away_team,home_score,away_score,tournament,country,neutral,home_rank,country_full
0,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,False,59,Bolivia
1,Brazil,Mexico,1,1,Friendly,Brazil,False,8,Brazil
2,Ecuador,Venezuela,5,0,FIFA World Cup qualification,Ecuador,False,35,Ecuador
3,Guinea,Sierra Leone,1,0,Friendly,Guinea,False,65,Guinea
4,Paraguay,Argentina,1,3,FIFA World Cup qualification,Paraguay,False,67,Paraguay
5,Peru,Colombia,0,1,FIFA World Cup qualification,Peru,False,70,Peru
6,Zimbabwe,Eswatini,2,0,Friendly,Zimbabwe,False,50,Zimbabwe


In [0]:
# Check for null values and drop them if necessary
merge_df.isnull().any()

home_team       False
away_team       False
home_score      False
away_score      False
tournament      False
country         False
neutral         False
home_rank       False
country_full    False
dtype: bool

**RESULT**: No nulll values

# Feature engineering

We can merge both tables to get the data we need, then create a few custom columns:
1. Turn the tournament type into categorical data
2. Create home team rank, and away team rank columns

In [0]:
merge_df.tournament.unique()

array(['FIFA World Cup qualification', 'Friendly', 'CFU Caribbean Cup',
       'Merdeka Tournament', 'South Pacific Games',
       'African Cup of Nations', 'AFC Asian Cup qualification',
       'CFU Caribbean Cup qualification', 'Copa América',
       'UEFA Euro qualification',
       'United Arab Emirates Friendship Tournament', 'UNCAF Cup',
       'Korea Cup', 'Gold Cup', 'USA Cup', 'Millennium Cup', "King's Cup",
       'Windward Islands Tournament', 'Gulf Cup',
       'Cyprus International Tournament',
       'Malta International Tournament', 'Gold Cup qualification',
       'UAFA Cup', 'AFF Championship', 'SAFF Cup', "Prime Minister's Cup",
       'Confederations Cup', 'AFC Asian Cup', 'Nehru Cup', 'COSAFA Cup',
       'CECAFA Cup', 'EAFF Championship',
       'AFC Challenge Cup qualification',
       'African Cup of Nations qualification', 'Copa del Pacífico',
       'Oceania Nations Cup', 'African Nations Championship',
       'Intercontinental Cup'], dtype=object)

In [0]:
tourn_types=[merge_df.tournament.unique()]

In [0]:
# Output unique items from a column in to an array.
# In this case the unique items are the different tournament types

typesss = merge_df.tournament.unique()
qq = []
for x in typesss:
    if x not in qq:
        qq.append(x)
print(qq)

['FIFA World Cup qualification', 'Friendly', 'CFU Caribbean Cup', 'Merdeka Tournament', 'South Pacific Games', 'African Cup of Nations', 'AFC Asian Cup qualification', 'CFU Caribbean Cup qualification', 'Copa América', 'UEFA Euro qualification', 'United Arab Emirates Friendship Tournament', 'UNCAF Cup', 'Korea Cup', 'Gold Cup', 'USA Cup', 'Millennium Cup', "King's Cup", 'Windward Islands Tournament', 'Gulf Cup', 'Cyprus International Tournament', 'Malta International Tournament', 'Gold Cup qualification', 'UAFA Cup', 'AFF Championship', 'SAFF Cup', "Prime Minister's Cup", 'Confederations Cup', 'AFC Asian Cup', 'Nehru Cup', 'COSAFA Cup', 'CECAFA Cup', 'EAFF Championship', 'AFC Challenge Cup qualification', 'African Cup of Nations qualification', 'Copa del Pacífico', 'Oceania Nations Cup', 'African Nations Championship', 'Intercontinental Cup']


In [0]:
# This section I have used to generate random integers corresponding with each type of tournament

d = {ni: indi for indi, ni in enumerate(set(qq))}
tc = [d[ni] for ni in qq]

In [0]:
print (tc)

[23, 6, 15, 29, 32, 21, 7, 4, 2, 9, 0, 18, 10, 22, 17, 5, 33, 36, 31, 26, 1, 11, 3, 13, 24, 16, 20, 35, 27, 8, 28, 19, 14, 12, 37, 30, 34, 25]


We still need to get the ranks for the away teams. They are required for our predictive analysis.

In [0]:
merge_df = merge_df.merge(ranking, left_on = ['away_team'],
                        right_on = ['country_full'], how = 'inner')

In [0]:
merge_df.head(3)

,home_team,away_team,home_score,away_score,tournament,country,neutral,home_rank,country_full_x,rank,country_full_y,country_abrv,total_points,previous_points,rank_change,cur_year_avg,cur_year_avg_weighted,last_year_avg,last_year_avg_weighted,two_year_ago_avg,two_year_ago_weighted,three_year_ago_avg,three_year_ago_weighted,confederation,rank_date
0,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,False,59,Bolivia,22,Uruguay,URU,0.0,47,-6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-08-08
1,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,False,59,Bolivia,15,Uruguay,URU,0.0,45,7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-09-23
2,Bolivia,Uruguay,3,1,FIFA World Cup qualification,Bolivia,False,59,Bolivia,17,Uruguay,URU,0.0,51,-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONMEBOL,1993-10-22


Now we rename the NEW rank column to away_team_rank, and drop unnecessary columns once more.

In [0]:
merge_df.rename(columns = {'rank' : 'away_rank'}, inplace = True)
print(merge_df.columns)

Index(['home_team', 'away_team', 'home_score', 'away_score', 'tournament',
       'country', 'neutral', 'home_rank', 'country_full_x', 'away_rank',
       'country_full_y', 'country_abrv', 'total_points', 'previous_points',
       'rank_change', 'cur_year_avg', 'cur_year_avg_weighted', 'last_year_avg',
       'last_year_avg_weighted', 'two_year_ago_avg', 'two_year_ago_weighted',
       'three_year_ago_avg', 'three_year_ago_weighted', 'confederation',
       'rank_date'],
      dtype='object')


In [0]:
merge_df.tournament.unique()

array(['FIFA World Cup qualification', 'Copa América', 'Friendly',
       'Korea Cup', 'Gold Cup', 'Cyprus International Tournament',
       'UEFA Euro qualification', 'Copa del Pacífico',
       'Merdeka Tournament', 'South Pacific Games',
       'African Cup of Nations', 'African Cup of Nations qualification',
       'COSAFA Cup', 'CECAFA Cup', "Prime Minister's Cup",
       'CFU Caribbean Cup qualification', 'Windward Islands Tournament',
       'UAFA Cup', 'UNCAF Cup', 'AFC Asian Cup qualification',
       'Oceania Nations Cup', "King's Cup", 'Gulf Cup',
       'United Arab Emirates Friendship Tournament',
       'African Nations Championship', 'Malta International Tournament',
       'Intercontinental Cup', 'AFF Championship', 'AFC Asian Cup',
       'USA Cup', 'Confederations Cup', 'Millennium Cup',
       'CFU Caribbean Cup', 'Gold Cup qualification', 'SAFF Cup',
       'Nehru Cup', 'EAFF Championship',
       'AFC Challenge Cup qualification'], dtype=object)

In [0]:
## Dropping unnecessary columns

merge_df.drop(columns = ['country_abrv', 'country', 'total_points', 'country_full_x', 'country_full_y', 'previous_points', 'rank_change', 
                        'cur_year_avg', 'cur_year_avg_weighted', 'last_year_avg', 'last_year_avg_weighted',
                        'two_year_ago_avg', 'two_year_ago_weighted', 'three_year_ago_avg', 
                        'three_year_ago_weighted', 'confederation', 'rank_date'], axis = 1, inplace = True)

In [0]:
## Creating 'friendly' column that displays 1 or 0 depending on whether or not a match is friendly
## PERSONAL NOTE: This replace function is like a manual version of label encoding, the code for
## label encoding is shown at the end of this list

#This next line replaces everything that isn't 'friendly' with a '0'
merge_df['friendly_y_n'] = merge_df['tournament'].replace(['FIFA World Cup qualification', 'Copa América', 'Korea Cup', 'Gold Cup', 'Cyprus International Tournament',
       'UEFA Euro qualification', 'Copa del Pacífico',
       'Merdeka Tournament', 'South Pacific Games',
       'African Cup of Nations', 'African Cup of Nations qualification',
       'COSAFA Cup', 'CECAFA Cup', "Prime Minister's Cup",
       'CFU Caribbean Cup qualification', 'Windward Islands Tournament',
       'UAFA Cup', 'UNCAF Cup', 'AFC Asian Cup qualification',
       'Oceania Nations Cup', "King's Cup", 'Gulf Cup',
       'United Arab Emirates Friendship Tournament',
       'African Nations Championship', 'Malta International Tournament',
       'Intercontinental Cup', 'AFF Championship', 'AFC Asian Cup',
       'USA Cup', 'Confederations Cup', 'Millennium Cup',
       'CFU Caribbean Cup', 'Gold Cup qualification', 'SAFF Cup',
       'Nehru Cup', 'EAFF Championship',
       'AFC Challenge Cup qualification'], '0', inplace = True)


#This next line replaces 'friendly' with a '1'
merge_df['friendly_y_n'] = merge_df['tournament'].replace(['Friendly'], '1', inplace = True)


# Personal Note #
# Sometimes, datatypes are messed up in the dataframe (ie they look like strings but are not). 
# When that happens, you need to add 'str' modifier before the replace function.
# Use:

# df['BrandName'] = df['BrandName'].str.replace(['ABC', 'AB'], 'A')




In [0]:
#Confirmation of cardinality for the 'tournament' column
merge_df.tournament.unique()

array(['0', '1'], dtype=object)

In [0]:
# Personal Note #
# Here is a function you can use for label encoding. Godspeed, Maloba!

# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# df['tournament'] = le.fit_transform(df['tournament'])

In [0]:
# Personal Note #
# Here's another nifty way to do it using a dicrionary with key:value pairs

# dictionary_name = {"column_1":     {"old_value": "new value", "old_value": "new value"},
#                 "column_2": {"four": 4, "six": 6, "five": 5, "eight": 8,
#                                   "two": 2, "twelve": 12, "three":3 }}

# df_name.replace(dictionary_name, inplace=True)
# df_name.head()

In [0]:
## Our final dataframe, ready for analysis


merge_df.drop(columns = ['friendly_y_n'], axis = 1, inplace = True)
merge_df.head(5)

,home_team,away_team,home_score,away_score,tournament,neutral,home_rank,away_rank
0,Bolivia,Uruguay,3,1,0,False,59,22
1,Bolivia,Uruguay,3,1,0,False,59,15
2,Bolivia,Uruguay,3,1,0,False,59,17
3,Bolivia,Uruguay,3,1,0,False,59,18
4,Bolivia,Uruguay,3,1,0,False,59,17


# EDA

In [0]:
import pandas_profiling as pp
pp.ProfileReport(merge_df)

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,8
Number of observations,247991
Total Missing (%),0.0%
Total size in memory,15.4 MiB
Average record size in memory,65.0 B
Numeric,4
Categorical,3
Boolean,1
Date,0
Text (Unique),0
Rejected,0


In [0]:
# --- PERSONAL NOTE --- #
# Here's a function you can use to  check VIF (Variance Inflation Factor), and drop variables whose VIF is more than 5.
# Remember: INDEPENDENT VARIABLES should not have high collinearity. I mean, are they truly independent if they are?

# from statsmodels.stats.outliers_influence import variance_inflation_factor    

# def calculate_vif_(X, thresh=100):
# cols = X.columns
# variables = np.arange(X.shape[1])
# dropped=True
# while dropped:
#     dropped=False
#     c = X[cols[variables]].values
#     vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]

#     maxloc = vif.index(max(vif))
#     if max(vif) > thresh:
#         print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
#         variables = np.delete(variables, maxloc)
#         dropped=True

# print('Remaining variables:')
# print(X.columns[variables])
# return X[cols[variables]]



In [0]:
## --UPDATED, CORRECTED VERSION OR VIF CHECK-- ##


# from statsmodels.stats.outliers_influence import variance_inflation_factor    
# def calculate_vif_(X, thresh):
#   cols = X.columns
#   variables = np.arange(X.shape[1])
#   dropped=True
#   while dropped:
#       dropped=False
#       c = X[cols[variables]].values
#       vif = [variance_inflation_factor(c, ix) for ix in np.arange(c.shape[1])]
#       maxloc = vif.index(max(vif))
#       if max(vif) > thresh:
#           print('dropping \'' + X[cols[variables]].columns[maxloc] + '\' at index: ' + str(maxloc))
#           variables = np.delete(variables, maxloc)
#           dropped=True
#   print('Remaining variables:')
#   print(X.columns[variables])
#   return X[cols[variables]]

#   # Calling the function
#   calculate_vif_('dataframe containing your independent variable' ,  1)

# Predictive analysis

There will be 2 models

1.   Predict how many goals the home team scores.
2.   Predict how many goals the away team scores.



In [0]:
# Multicollinearity & VIF checks

corr = merge_df.corr()
print ('Correlation matrix')
print ('....')
print(corr)

# ------
print ('------------------------------------')
print ('VIF matrix')
print ('....')
pd.DataFrame(np.linalg.inv(merge_df.corr().values), index = corr.index, columns=corr.columns)


Correlation matrix
....
            home_score  away_score   neutral  home_rank  away_rank
home_score    1.000000   -0.133919  0.012780  -0.040099   0.306575
away_score   -0.133919    1.000000  0.086808   0.144491  -0.145258
neutral       0.012780    0.086808  1.000000   0.064096   0.085021
home_rank    -0.040099    0.144491  0.064096   1.000000   0.489355
away_rank     0.306575   -0.145258  0.085021   0.489355   1.000000
------------------------------------
VIF matrix
....


,home_score,away_score,neutral,home_rank,away_rank
home_score,1.166734,0.044635,0.004837,0.278873,-0.488087
away_score,0.044635,1.101675,-0.102686,-0.298076,0.300939
neutral,0.004837,-0.102686,1.017597,0.000223,-0.103025
home_rank,0.278873,-0.298076,0.000223,1.469067,-0.847708
away_rank,-0.488087,0.300939,-0.103025,-0.847708,1.616939


A typical threshold for VIF is 5, and that is the same threshold applied here. A score  of 4 would be problematic as well. None of the VIF values (seen in the VIF matrix above) are above this threshold. In fact they are way lower, which means that the extent of correlation between our predictors (independent variables) is very low. 

This is a good thing. If your predictors are highly correlated, then are they truly 'independent' variables?

## Using chosen models to predict home score

In [0]:
# Predict Home Score for different teams

# Set dependent and independent variables

x_home = np.array(merge_df[['home_rank', 'away_rank', 'tournament']])
y_home = merge_df['home_score']

print(x_home.shape)
print(y_home.shape)

(247991, 3)
(247991,)


In [0]:
# Fitting of model to the dataset

reg_line_home = LinearRegression()
reg_line_home.fit(x_home, y_home)



LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
#Establish training and testing sets

x_home_train, x_home_test, y_home_train, y_home_test = train_test_split(x_home, y_home, test_size = 0.2, random_state = 0)

In [0]:
# Fit polynomial regression to the data

poly_reg_home = PolynomialFeatures(degree = 5)
x_poly_home = poly_reg_home.fit_transform(x_home)

In [0]:
# Fit linear regression to our polynomial regression model

lin_reg_home = LinearRegression()
lin_reg_home.fit(x_poly_home, y_home)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

---
Let's test the accuracy of our chosen model against the base model

For our test,

Home rank: 20

Away rank: 4

Friendly: 1

In [0]:
# predict using linear regression
lin_pred_home = reg_line_home.predict([[20, 4, 1]])

print("Linear Prediction: %d" %lin_pred_home)
print("Linear Prediction (accurate):", lin_pred_home)

print ('-------')

# predict using polynomial regression
poly_pred_home = lin_reg_home.predict(poly_reg_home.fit_transform([[20, 4, 1]]))

print("Polynomial Prediction: %d" %poly_pred_home)
print("Polynomial Prediction (accurate):", poly_pred_home)

Linear Prediction: 0
Linear Prediction (accurate): [0.99320826]
-------
Polynomial Prediction: 1
Polynomial Prediction (accurate): [1.00860439]


The linear regression (base) model predicted the home team scoring 0 goals.

The polynomial regression model predicted 1 goal.

I'd attribute this to a rounding error. Looking at the accurate values, the values are 0.99 and 1.01 for linear and polynomial respectively. 

We can say that both models predicted a home team score of 1 goal.

Now for a test with data whose outcome we already know...


In [0]:
# Here's what we'll be testing
merge_df.iloc[2650]

home_team       Spain
away_team     Uruguay
home_score          2
away_score          0
tournament          1
neutral         False
home_rank           8
away_rank          34
Name: 2650, dtype: object

In [0]:
# predict using linear regression
lin_pred_home = reg_line_home.predict([[8, 34, 1]])

print("Linear Prediction: %d" %lin_pred_home)
print("Linear Prediction (accurate):", lin_pred_home)

print ('-------')

# predict using polynomial regression
poly_pred_home = lin_reg_home.predict(poly_reg_home.fit_transform([[20, 4, 1]]))

print("Polynomial Prediction: %d" %poly_pred_home)
print("Polynomial Prediction (accurate):", poly_pred_home)

Linear Prediction: 1
Linear Prediction (accurate): [1.56512689]
-------
Polynomial Prediction: 1
Polynomial Prediction (accurate): [1.00860439]


Both models predicted a score of 1, which is wrong. The correct home score for this test match is 2 goals. More work can be done to tweak our model.

In [0]:
# Test model accuracy using RMSE

regressor_home = LinearRegression()
regressor_home.fit(x_home_train, y_home_train)

y_predict_home = regressor_home.predict(x_home_test)

print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_home_test, y_predict_home)))

Root Mean Squared Error: 1.7146593746716312




I don't think correlation is an issue here. correlation implies that the input data somehow affects the output data, and in this case, team rank can very much be used as a predictor for how well a team will do.  

I also don't think that the number of categories is an issue here,since they are so few.

As far as improving the RMSE score, one of the only options left with best chances to affect it the way we want, is to go through the cleaning of the data again.

## Cross validation (k-folds)


In [0]:
folds = KFold(n_splits = 5)

# create array to store the RMSE values of each fold
RMSES_home = []

# create a counter
count_home = 1

for train_index, test_index in folds.split(X_home):
    print("\nTraining model " + str(count_home))
    
    X_home_train, X_home_test = X_home[train_index], X_home[test_index]
    y_home_train, y_home_test = y_home[train_index], y_home[test_index]
    
    # fit a regression model accordingly
    reg_home = LinearRegression()
    reg_home.fit(X_home_train, y_home_train)
    
    # assess the accuracy of the model
    y_home_pred = reg_home.predict(X_home_test)
    
    rmse_value_home = np.sqrt(metrics.mean_squared_error(y_home_test, y_home_pred))
    RMSES_home.append(rmse_value_home)
    
    print('Model ' + str(count_home) + ' Root Mean Squared Error:',rmse_value_home)
    count_home += 1

     


Training model 1
Model 1 Root Mean Squared Error: 1.4898928174827937

Training model 2
Model 2 Root Mean Squared Error: 1.3505421776235278

Training model 3
Model 3 Root Mean Squared Error: 1.103444191894206

Training model 4
Model 4 Root Mean Squared Error: 1.4254045144956964

Training model 5
Model 5 Root Mean Squared Error: 2.862297838186504


In [0]:
print('Average RMSE value', np.mean(RMSES_home))

Average RMSE value 1.6463163079365457


The RMSE measurement when implemented using k-folds is a fairer metric. And that is why the mean of 1.65 is troubling.

It is just a smidge lower than the 1.7 we got above. In our football context here, it means our scores can potentially be off by 2 goals. That severely affects Mchezopesa's ability to offer betting odds to customers, that will favor the business.

The fixes described above can be applied to try and better our model.

----

## Residual plots and heteroscedatisticity tests

In [0]:
# set prediction
home_reg = LinearRegression()
home_reg.fit(X_home_train, y_home_train)
home_predict = home_reg.predict(X_home_test)

In [0]:
# creating residuals
residuals_home = np.subtract(home_predict, y_home_test)
residuals_home.head()

198393    0.530455
198394    0.551487
198395    0.551487
198396    0.551487
198397    0.562003
Name: home_score, dtype: float64

## Predict Away Scores



In [0]:
#Establish predictors and predicted fields

X_away = np.array(merge_df[['home_rank', 'away_rank', 'tournament']])
y_away = merge_df['away_score']
print(X_away.shape)
print(y_away.shape)

(247991, 3)
(247991,)


In [0]:
# fitting the Linear Regression model
reg_line_away = LinearRegression()
reg_line_away.fit(X_away, y_away)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
# getting our training and testing sets
X_train_away,  X_test_away, y_train_away, y_test_away = train_test_split(X_away, y_away, test_size = 0.2, random_state = 0)

In [0]:
# fitting a polynomial regression model
poly_reg_away = PolynomialFeatures(degree = 5)
X_poly_away = poly_reg_away.fit_transform(X_away)

In [0]:
# fitting a linear regression to our polynomial regression
lin_reg_away = LinearRegression()
lin_reg_away.fit(X_poly_away, y_away)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

Here I'll repreat the tests done with the first model, except that now we're trying to predict away scores.

In [0]:
# Here's what we'll be testing
merge_df.iloc[2650]

home_team       Spain
away_team     Uruguay
home_score          2
away_score          0
tournament          1
neutral         False
home_rank           8
away_rank          34
Name: 2650, dtype: object

In [0]:
# predict using linear regression
lin_pred_away = reg_line_away.predict([[8, 34, 1]])
print("Linear Prediction: %d" %lin_pred_away)
print("Linear Prediction (accurate):", lin_pred_away)

# predict using polynomial regression
poly_pred_away = lin_reg_away.predict(poly_reg_away.fit_transform([[8, 34, 1]]))
print("Polynomial Prediction: %d" %poly_pred_away)
print("Polynomial Prediction (accurate):", poly_pred_away)

Linear Prediction: 0
Linear Prediction (accurate): [0.83879383]
Polynomial Prediction: 0
Polynomial Prediction (accurate): [0.6120657]


Both models returned a correct score. However, I'm still not sure how it handles rounding.

While the accuracy here may be better, more can be done to reduce errors. Our cross-validation below will tell us just how off the model is.

## Cross validation of model 2

In [0]:
regressor_away = LinearRegression()
regressor_away.fit(X_train_away, y_train_away)

y_predict_away = regressor_away.predict(X_test_away)

print("Root Mean Squared Error:", np.sqrt(metrics.mean_squared_error(y_test_away, y_predict_away)))

Root Mean Squared Error: 1.085633761981341


In [0]:
# create array to store the RMSE values of each fold
RMSES_away = []

# create a counter
count_away = 1

for train_index, test_index in folds.split(X_away):
    print("\nTraining model " + str(count_away))
    
    X_away_train, X_away_test = X_away[train_index], X_away[test_index]
    y_away_train, y_away_test = y_away[train_index], y_away[test_index]
    
    # fit a regression model accordingly
    reg_away = LinearRegression()
    reg_away.fit(X_away_train, y_away_train)
    
    # assess the accuracy of the model
    y_away_pred = reg_away.predict(X_away_test)

    rmse_value_away = np.sqrt(metrics.mean_squared_error(y_away_test, y_away_pred))
    RMSES_away.append(rmse_value_away)
    
    print('Model ' + str(count_away) + ' Root Mean Squared Error:',rmse_value_away)
    count_away += 1


Training model 1
Model 1 Root Mean Squared Error: 1.1692334491491383

Training model 2
Model 2 Root Mean Squared Error: 1.1433767797579573

Training model 3
Model 3 Root Mean Squared Error: 1.179987281695326

Training model 4
Model 4 Root Mean Squared Error: 1.0472729995410368

Training model 5
Model 5 Root Mean Squared Error: 1.008805041684027


In [0]:
# average RMSE values from the above folds
np.mean(RMSES_away)

1.1097351103654973

The RMSE is lower for this model than for the first one. Much lower. This is a good result. Either way, there's always room for improvement.

The aforementioned measures can be applied to improve on this metric

---

## Residual plots & Heteroscedasticity for the away score model



In [0]:
# set prediction
away_reg = LinearRegression()
away_reg.fit(X_train_away, y_train_away)
away_predict = away_reg.predict(X_test_away)

In [0]:
residuals_away = np.subtract(away_predict, y_test_away)

# preview the  residuals
residuals_away.head()

52496    -0.809699
185763    0.747351
110626   -0.061532
143964   -0.120860
183510    0.233691
Name: away_score, dtype: float64